In [1]:
%config Completer.use_jedi = False                                       
import warnings                                                             
warnings.filterwarnings(action="ignore")
import numpy as np
import pandas as pd                                                         
import matplotlib.pyplot as plt                                            
import matplotlib as mpl                                                    
mpl.rcParams['axes.unicode_minus'] = False                                  
#plt.rcParams('font.family') = 'RIDIBatang'                          
#plt.rcParams('font.size') = 16                                             
import matplotlib.font_manager as fm
font = 'C:\\Windows\\Fonts\\RIDIBatang.otf'
fontprop = fm.FontProperties(fname=font, size=16).get_name()
plt.rc('font', family = 'RIDIBatang')
plt.rc('font', size = 16)
import seaborn as sns                                                       
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

***
베르누이 나이브베이즈(Bernoulli NaiveBayes)
***
분류 데이터의 특징이 0 또는 1로 표현되었을 때 데이터의 출현 여부에 따라서 0 또는 1로 구분되는 데이터에 사용한다.  

***
이메일 제목과 레이블(스팸여부)를 활용해 베르누이 나이브베이즈 분류로 스팸메일을 분류한다.
***

***
데이터 획득
***
간단한 스팸 메일 분류를 위해 다음과 같이 이메일 제목과 스팸 메일 레이블이 붙어있는 데이털르 사용한다.  
email title : 이메일 제목, spam : 스팸 메일 여부(True -> 스팸 메일, False -> 스팸 메일이 아님)

In [2]:
# 학습 데이터
email_list = [
    {'email title' : 'free game only today', 'spam' : True},
    {'email title' : 'cheapest flight deal', 'spam' : True},
    {'email title' : 'limited time offer only today only today', 'spam' : True},
    {'email title' : 'today meeting schedule', 'spam' : False},
    {'email title' : 'your flight schedule attached', 'spam' : False},
    {'email title' : 'your credit card statement', 'spam' : False}
]

# 테스트 데이터
test_email_list = [
    {'email title' : 'free flight offer', 'spam' : True},
    {'email title' : 'hey traveler free flight deal', 'spam' : True},
    {'email title' : 'limited free game offer', 'spam' : True},
    {'email title' : 'today flight schedule', 'spam' : False},
    {'email title' : 'your credit card attached', 'spam' : False},
    {'email title' : 'free credit card offer only today', 'spam' : False}
]

In [3]:
#학습 데이터 준비
df = pd.DataFrame(email_list)
df

,email title,spam
0,free game only today,True
1,cheapest flight deal,True
2,limited time offer only today only today,True
3,today meeting schedule,False
4,your flight schedule attached,False
5,your credit card statement,False


In [4]:
# 학습 데이터 다듬기
# Scikit-learn의 베르누이 나이브베이즈 분류기는 숫자 데이터(0과 1)만 다루기때문에
# True를 1, false를 0으로 치환한 'label'파생 변수를 추가한다.
df['label'] = df.spam.map({True : 1, False : 0})
df

,email title,spam,label
0,free game only today,True,1
1,cheapest flight deal,True,1
2,limited time offer only today only today,True,1
3,today meeting schedule,False,0
4,your flight schedule attached,False,0
5,your credit card statement,False,0


***
학습에 사용할 데이터(피쳐)와 레이블로 분리한다.
***

In [5]:
x = df['email title'] # 피쳐
x

0                        free game only today
1                        cheapest flight deal
2    limited time offer only today only today
3                      today meeting schedule
4               your flight schedule attached
5                  your credit card statement
Name: email title, dtype: object

In [6]:
y = df['label'] # 레이블
y

0    1
1    1
2    1
3    0
4    0
5    0
Name: label, dtype: int64

***
이메일 제목으로 학습을 진행하고, 레이블을 사용해서 스팸 메일 여부를 판단한다.
***
베르누이 나이브베이즈의 입력 데이터는 고정된(동일한) 크기의 벡터이어야 한다. 

In [7]:
# 모든 데이터에 출련한 단어 갯수 만큼의 크기를 가지는 벡터를 만들고, 고정된 벡터로 표현하기 위해 import 한다.
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
string = 'free game only today cheapest flight deal limited time offer only today only today today meeting' \
        'schedule your flight schedule attached your credit card statement'
print(string)
string = set(string.split(' '))
print(len(string))
string = list(string)
print(string)
string.sort()
print(string)

free game only today cheapest flight deal limited time offer only today only today today meetingschedule your flight schedule attached your credit card statement
17
['flight', 'attached', 'deal', 'card', 'statement', 'time', 'only', 'credit', 'today', 'cheapest', 'limited', 'meetingschedule', 'offer', 'free', 'your', 'schedule', 'game']
['attached', 'card', 'cheapest', 'credit', 'deal', 'flight', 'free', 'game', 'limited', 'meetingschedule', 'offer', 'only', 'schedule', 'statement', 'time', 'today', 'your']


In [9]:
#CountVectorizer 객체는 문자열에 출현한 모든 단어를 오름차순으로 정렬해 단어의 위치로 행렬을 만든다.
#특정 단어가 출현할 경우 출현한 단어의 갯수를 리턴하고, 출현하지 않으면 0을 리턴한다.
#binary 옵션의 기본값은 None으로 출현한 단어의 갯수를 리턴하고,
#True로 변경하면 같은 단어가 여러번 출현하더라도 무조건 1로 리턴한다.
cv = CountVectorizer(binary=True) #CountVectorizer 객체를 만든다.
#x_train = cv.fit(x)#CountVectorizer 객체를 학습시킨다.
#x_train = cv.transform(x) #CountVectorizer 객체의 학습 결과를 적용한다.
x_train = cv.fit_transform(x) # CountVectorizer 객체를 학습하고 결과를 적용한다.
#print(x_train)
encoding = x_train.toarray() #CountVectorizer 객체의 학습 결과를 numpy 배열로 변환한다.
print(encoding)
print(type(encoding))

[[0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0]
 [0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0]
 [1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1]]
<class 'numpy.ndarray'>


***
위의 numpy 배열에서 볼 수 있듯이 이메일 제목에서 총 17개의 단어가 발견되어  각 이메일 제목이 17개 크기의 벡터로 표현(인코딩)된 것을 확인할 수 있다.  
binary=True 옵션을 지정해서 베르누이 나이브베이즈에서 사용하기위하여 이메일 제목에 중복되어 출현한 단어가 있더라도  
출현한 횟수로 표현되는 것이 아니고 단순히 1로 표현된 것도 알 수 있다.  


In [10]:
#inverse_transform() 메소드로 고정된 크기의 벡터에 포함된 단어를 확인할 수 있다.
for string in cv.inverse_transform(x_train):
    print(string)

['free' 'game' 'only' 'today']
['cheapest' 'flight' 'deal']
['only' 'today' 'limited' 'time' 'offer']
['today' 'meeting' 'schedule']
['flight' 'schedule' 'your' 'attached']
['your' 'credit' 'card' 'statement']


In [11]:
#get_feature_names() 메소드로 고정된 벡터의 각 열(피쳐)이 어떤 단어를 의미하는지 확인할 수 있다.
print(cv.get_feature_names())

['attached', 'card', 'cheapest', 'credit', 'deal', 'flight', 'free', 'game', 'limited', 'meeting', 'offer', 'only', 'schedule', 'statement', 'time', 'today', 'your']


***
베르누이 나이브베이즈 모델 학습하기
***
Scikit-learn의 베르누이 나이브베이즈 분류기는 기본적으로 라플라스 스무딩을 지원하므로 학습 데이터에 없던 단어가 테스트 데이터에 있어도 분류가 진행된다.

***
라플라스 스무딩(Laplace Smoothing)  
***
0이라는 수는 곱셈과 나눗셈을 무력화시키는 값이므로 그전에 아무리 의미있는 값이 도출된다 하더라도 마지막에 0을 곱해버린면 값은 0이 나오게 된다.  
이런 경우가 상당히 빈번하기떄문에 0이 아닌 최소값으로 보정을 하는데 이를 라플라스 스무딩이라고 한다.  

In [12]:
# 베르누이 나이브베이즈 모델을 사용하기 위해 import한다.
from sklearn.naive_bayes import BernoulliNB

In [13]:
# 베르누이 나이브베이즈 모델을 만들고 학습시킨다.
bnb = BernoulliNB().fit(x_train, y)

***
테스트 데이터 준비하고 다듬기
***

In [14]:
# Scikit-learn의 베르누이 나이브베이즈 분류기는 숫자 데이터(0과 1)만 다루기때문에
# True를 1, false를 0으로 치환한 'label'파생 변수를 추가한다.
test_df = pd.DataFrame(test_email_list)
test_df['label'] = df.spam.map({True : 1, False : 0})
test_df

,email title,spam,label
0,free flight offer,True,1
1,hey traveler free flight deal,True,1
2,limited free game offer,True,1
3,today flight schedule,False,0
4,your credit card attached,False,0
5,free credit card offer only today,False,0


***
학습에 사용할 데이터(피쳐)와 레이블로 분리한다.
***

In [15]:
x_test = test_df['email title'] # 피쳐
x_test

0                    free flight offer
1        hey traveler free flight deal
2              limited free game offer
3                today flight schedule
4            your credit card attached
5    free credit card offer only today
Name: email title, dtype: object

In [16]:
y_test = test_df['label'] # 레이블
y_test

0    1
1    1
2    1
3    0
4    0
5    0
Name: label, dtype: int64

***
모델 테스트
***

In [17]:
# CountVectorizer 객체는 학습 데이터를 다듬을때 이미 학습을 시켰으므로 테스트 시에는 적용만 시키면 된다.
x_test_apply = cv.transform(x_test)

In [18]:
#predict() 메소드의 인수로 테스트 데이터의 피쳐를 넘겨서 예측치를 계산한다.
predict = bnb.predict(x_test_apply)
print(predict)
#accuracy_score() 메소드의 인수로 테스트 데이터의 레이블(실제값, 실측치, 결과)과 예측값(예측 결과값)을 넘겨서 정확도를 계산한다.
accuracy = accuracy_score(y_test, predict)
print('정확도 -> {:6.2%}'.format(accuracy))

[1 1 1 0 0 1]
정확도 -> 83.33%


In [19]:
#confusion_matrix() 메소드의 인수로 테스트 데이터의 레이블과 예측값을 넘겨서 혼동 행렬을 출력한다.
print(confusion_matrix(y_test, predict))

[[2 1]
 [0 3]]


In [20]:
# classification_report() 메소드ㅢ 인수로 테스트 데이터의 레이블과 예측값을 넘겨서 분류 리포트를 출력한다.
print(classification_report(y_test, predict))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.75      1.00      0.86         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6



In [21]:
pd.DataFrame({'실제값' : y_test, '예측값' : predict})

,실제값,예측값
0,1,1
1,1,1
2,1,1
3,0,0
4,0,0
5,0,1
